# Aula05.Ex01 - Exemplo - Cubo - Transformação Geométrica 3D

### Primeiro, vamos importar as bibliotecas necessárias.

In [129]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
import time

### Inicializando janela

In [130]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

### Shaders

Note que agora usamos vec3, já que estamos em 3D.

In [131]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

In [132]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [133]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [134]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [135]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [136]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [137]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [138]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Funcoes gerais

In [139]:
#variaveis
#matriz identidade
identity = np.array([1.0,  0.0, 0.0,   0, 
                    0.0,    1.0,   0.0, 0, 
                    0.0,    0.0,   1.0, 0, 
                    0.0,    0.0,   0.0, 1.0], np.float32)
total_vertices = 0 #numero total de vertices
vertices_list = [] #lista de coordenadas de vertices da forma (x,y,z)

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

#soma o valor das 2 coordenadas
#coord1: list[3]
#coord2: list[3]
def sum_coord(coord1, coord2):
    return [coord1[0]+coord2[0], coord1[1]+coord2[1], coord1[2]+coord2[2]]


#retorna matriz de transicao
def traslation_matrix(translate_coord):
    return np.array([1.0,  0.0, 0.0, translate_coord[0], 
                    0.0,    1.0,   0.0, translate_coord[1], 
                    0.0,    0.0,   1.0, translate_coord[2], 
                    0.0,    0.0,   0.0, 1.0], np.float32)


#retorna matriz de rotacao
def rotate_matrix(rot_coord,center_coord = identity):
    cos_x = math.cos(math.radians(rot_coord[0]))
    sin_x = math.sin(math.radians(rot_coord[0]))
    cos_y = math.cos(math.radians(rot_coord[1]))
    sin_y = math.sin(math.radians(rot_coord[1]))
    cos_z = math.cos(math.radians(rot_coord[2]))
    sin_z = math.sin(math.radians(rot_coord[2]))

    #matrizes
    mat_rotation_z = np.array([cos_z, -sin_z, 0.0, 0.0, 
                                sin_z,  cos_z, 0.0, 0.0, 
                                0.0,      0.0, 1.0, 0.0, 
                                0.0,      0.0, 0.0, 1.0], np.float32)
    mat_rotation_x = np.array([1.0,   0.0,    0.0, 0.0, 
                                0.0, cos_x, -sin_x, 0.0, 
                                0.0, sin_x,  cos_x, 0.0, 
                                0.0,   0.0,    0.0, 1.0], np.float32)
        
    mat_rotation_y = np.array([cos_y,  0.0, sin_y, 0.0, 
                                0.0,    1.0,   0.0, 0.0, 
                                -sin_y, 0.0, cos_y, 0.0, 
                                0.0,    0.0,   0.0, 1.0], np.float32)
    
    #multiplicacao de matrizes
    minus_center = [-1 * x for x in center_coord]
    temp = traslation_matrix(minus_center)
    temp = multiplica_matriz(mat_rotation_z,temp)
    temp = multiplica_matriz(mat_rotation_x,temp)
    temp = multiplica_matriz(mat_rotation_y,temp)
    return multiplica_matriz(traslation_matrix(center_coord),temp)


# #retorna matriz de scale
def scale_matrix(rot_coord, center_coord = identity):
    rot_translate = np.array([rot_coord[0], 0.0, 0.0, 0.0, 
                    0.0,  rot_coord[1], 0.0, 0.0, 
                    0.0,      0.0, rot_coord[2], 0.0, 
                    0.0,      0.0, 0.0, 1.0], np.float32)
    minus_center = [-1 * x for x in center_coord]
    temp = traslation_matrix(minus_center)
    temp = multiplica_matriz(rot_translate,temp)
    return multiplica_matriz(traslation_matrix(center_coord),temp)

### Classe do Objeto

In [140]:
class Object:
    #vertices = coordenadas dos vertices
    #cor = lista de cores
    #indice_cor = lista quantos vertices tem para cada cor
    #transform = matriz de transformacao
    #coord = coordenadas do centro
    def __init__(self,vertices = [], color = [[0,0,0,0]], num_vertice_color = [], coordinate = [0,0,0], transform = identity):
        global total_vertices,vertices_list
        self.inicial_vertice = total_vertices
        total_vertices += len(vertices)
        vertices_list += vertices

        self.vertices = vertices
        self.color = color
        self.num_vertice_color = num_vertice_color
        self.transform = transform
        self.coord = coordinate

    #retorna vertices transformados
    def get_vertices(self):
        self.transform = self.transform.reshape(4, 4)  # Converte o array 1D para 4x4
        vertices_transformados = []
        for v in self.vertices:
            v_h = np.array([v[0], v[1], v[2], 1], dtype=np.float32)  # Converte para coordenadas homogêneas
            v_t = np.dot(self.transform, v_h)  # Multiplica pela matriz de transformação
            vertices_transformados.append(v_t[:3])  # Remove o componente homogêneo (volta para [x, y, z])
        return vertices_transformados

    
    #retorna a coordenada do centro do objeto e a matriz transladada
    #translate_coord: float[3]
    def translate(self, translate_coord):
        self.coord = sum_coord(self.coord,translate_coord)
        self.transform = multiplica_matriz(traslation_matrix(translate_coord), self.transform)

    #retorna matriz rotacionada
    #rot_coord: float[3]
    def rotate(self, rot_coord, center_coord = None):
        if center_coord == None: center_coord = self.coord
        self.transform = multiplica_matriz(rotate_matrix(rot_coord,center_coord),self.transform)

    #retorna o matriz escalada
    #scale_coord: float[3]
    def scale(self,scale_coord, center_coord = None):
        if center_coord == None: center_coord = self.coord
        self.transform = multiplica_matriz(scale_matrix(scale_coord,center_coord), self.transform)

    #renderiza o objeto
    def render(self,transform = identity):
        #envia o matriz de transformacao para shader
        loc_transformation = glGetUniformLocation(program, "mat_transformation")
        glUniformMatrix4fv(loc_transformation, 1, GL_TRUE, multiplica_matriz(transform, self.transform))
        num_vertice = self.inicial_vertice
        for i in range(0,len(self.color)):
            #cor
            glUniform4f(loc_color, self.color[i][0], self.color[i][1], self.color[i][2], self.color[i][3])
            #desenha   
            glDrawArrays(GL_TRIANGLE_STRIP, num_vertice, self.num_vertice_color[i])
            num_vertice += self.num_vertice_color[i]




#classe para manipular varios objetos juntos
class Group:
    def __init__(self,objects = []):
        self.objects = objects
    
    def get_coord(self):
        coord_list = []
        for i in range(len(self.objects)):
            coord_list.append(self.objects[i].coord)
        return coord_list
    
    def set_coord(self,coord_list):
        for i in range(len(self.objects)):
            self.objects[i].coord = coord_list[i]
    
    def translate(self, translate_coord):
        for i in range(len(self.objects)):
            self.objects[i].translate(translate_coord)
    
    def rotate(self, rotate_coord, center_coord = [0,0,0]):
        coord_list = self.get_coord()
        for i in range(len(self.objects)):
            self.objects[i].coord = center_coord
            self.objects[i].rotate(rotate_coord)
        self.set_coord(coord_list)
    
    def scale(self, scale_coord, center_coord = [0,0,0]):
        coord_list = self.get_coord()
        for i in range(len(self.objects)):
            self.objects[i].coord = center_coord
            self.objects[i].scale(scale_coord)
        self.set_coord(coord_list)
    
    def render(self):
        for i in range(len(self.objects)):
            self.objects[i].render()
        

### Preparando dados para enviar a GPU

Até aqui, compilamos nossos Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [141]:
#retorna vertices para a esfera
#r = raio [x,y,z]
#num_sectors = qtd de sectors (longitude)
#num_stacks = qtd de stacks (latitude)
PI = 3.141592
def vertice_esfera(r = [1,1,1],num_sectors = 32,num_stacks = 32):

    # grid sectos vs stacks (longitude vs latitude)
    sector_step=(PI*2)/num_sectors # variar de 0 até 2π
    stack_step=(PI)/num_stacks # variar de 0 até π

    # Entrada: angulo de longitude, latitude, raio
    # Saida: coordenadas na esfera
    def F(u,v,r):
        x = r[0]*math.sin(v)*math.cos(u)
        y = r[1]*math.sin(v)*math.sin(u)
        z = r[2]*math.cos(v)
        return (x,y,z)

    # vamos gerar um conjunto de vertices representantes poligonos
    # para a superficie da esfera.
    # cada poligono eh representado por dois triangulos
    vertices_list = []
    for i in range(0,num_sectors): # para cada sector (longitude)
        for j in range(0,num_stacks): # para cada stack (latitude)
            
            u = i * sector_step # angulo setor
            v = j * stack_step # angulo stack
            
            un = 0 # angulo do proximo sector
            if i+1==num_sectors:
                un = PI*2
            else: un = (i+1)*sector_step
                
            vn = 0 # angulo do proximo stack
            if j+1==num_stacks:
                vn = PI
            else: vn = (j+1)*stack_step
            
            # vertices do poligono
            p0=F(u, v, r)
            p1=F(u, vn, r)
            p2=F(un, v, r)
            p3=F(un, vn, r)
            
            # triangulo 1 (primeira parte do poligono)
            vertices_list.append(p0)
            vertices_list.append(p2)
            vertices_list.append(p1)
            
            # triangulo 2 (segunda e ultima parte do poligono)
            vertices_list.append(p3)
            vertices_list.append(p1)
            vertices_list.append(p2)

    return vertices_list


#enemy
ear_vertice = [# Face 1 da Tetraedro (triangulo)
    (0.0, 0.5, 0),
    (-0.25, 0.0, +0.25),
    (+0.25, 0.0, +0.25),
    # Face 2 da Tetraedro (triangulo)
    (0.0, 0.5, 0),
    (-0.25, 0.0, +0.25),
    (0, 0.0, -0.25),
    # Face 3 da Tetraedro (triangulo)
    (0.0, 0.5, 0),
    (+0.25, 0.0, +0.25),
    (0, 0.0, -0.25),
    # Face 4 (base) da Tetraedro (triangulo)
    (-0.25, 0.0, +0.25),
    (+0.25, 0.0, +0.25),
    (0, 0.0, -0.25)
]
ear_left = Object(ear_vertice)
ear_right = Object(ear_vertice)
ear_left.scale([0.2,0.4,0.2])
ear_right.scale([0.2,0.4,0.2])
ear_left.translate([0.06,0.08,0])
ear_right.translate([-0.06,0.08,0])
ear_left_vertice = ear_left.get_vertices()
ear_right_vertice = ear_right.get_vertices()
enemy_vertice = ear_left_vertice+ear_right_vertice+vertice_esfera([0.15,0.15,0.15])
enemy = Object(enemy_vertice,
            [[0,0,0,1],[0,0,0,1],[0,0,0,1]], #modificado embaixo
            [12,12,len(enemy_vertice)-24],
            )


#chao
chao_altura = -0.3
chao_z = 0.5
chao_vertice = [(-1,-1,chao_z),(1,-1,chao_z),(-1,chao_altura,chao_z),(1,chao_altura,chao_z),(-1,1,chao_z),(1,1,chao_z),(-1,chao_altura,chao_z),(1,chao_altura,chao_z)]
chao = Object(chao_vertice,
            [[0.22,0.10,0,1],[0.16,0.33,0,1]],
            [4,4]
            )


#player
foot_left_vertice = vertice_esfera([0.13,0.05,0.07])
foot_right_vertice = vertice_esfera([0.13,0.05,0.07])
foot_left = Object(foot_left_vertice,
                    [[0.70,0.42,0,1]],
                    [len(foot_left_vertice)])
foot_right = Object(foot_right_vertice,
                    [[0.80,0.52,0,1]],
                    [len(foot_right_vertice)])
foot_left.translate([0.04,-0.15,0.08])
foot_right.translate([0.04,-0.15,-0.08])
foot_left.coord = [0,0,0]
foot_right.coord = [0,0,0]

body_vertice = vertice_esfera([0.15,0.15,0.15])
body = Object(body_vertice,
        [[0.96,0.94,0.24,1]],
        [len(body_vertice)]
        )


#transformacao do vertice_list para np.array
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)


### Partes que nao mudam

In [142]:
# Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot (buffer).

# Request a buffer slot from GPU
buffer_VBO = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)

# Abaixo, nós enviamos todo o conteúdo da variável vertices.

#Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO)

#Associando variáveis do programa GLSL (Vertex Shader) com nossos dados

#Primeiro, definimos o byte inicial e o offset dos dados.
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

#Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

# A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

# Outros parâmetros:

# * Definimos que possui <b> três </b> coordenadas
# * Que cada coordenada é do tipo float (GL_FLOAT)
# * Que não se deve normalizar a coordenada (False)

# Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

#Vamos pegar a localização da variável color para que possamos definir a cor em nosso laço da janela!
loc_color = glGetUniformLocation(program, "color")

#Nesse momento, nós exibimos a janela!
glfw.show_window(window)

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [143]:
# def key_event(window,key,scancode,action,mods):

# glfw.set_key_callback(window,key_event)

### Loop principal da janela.

In [144]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

#variaveis
#malha
is_Malha = False
p_pressed = False

#enemy
enemy_color = [[0.7,0.4,0,0,1],[0.8,0.45,0,1],[0.84,0.51,0,1]]
enemy_big_color = [[0.71,0.2,0,1],[0.81,0.2,0,1],[0.85,0.25,0,1]]
enemy_scale = 1
enemy_anim_frame = 90
enemy_anim_speed = 0.003
enemy_frame = 0

#player
IDLE = "Idle"
SQUAT = "Squat"
JUMP = "Jump"
WALK = "Walk"
state = IDLE

player_velocity = [0,0,0] #velocidade atual do player
player_x_velocity = 0.025
walk_frame = 0
is_Right = True
walk_rotate_sum = 0
walk_cicle_time = 30
walk_init_rot = 15
walk_force = 12

gravity = 0.004
jump_force = 0.08
jump_init_y = 0
jump_frame = 0
jump_total_frame = jump_force / gravity * 2
jump_left_up_rot = -15
jump_right_up_rot = 10
jump_up_anim_frame = 10
jump_rotate_sum_left = 0
jump_rotate_sum_right = 0



#transformacoes iniciais
enemy.rotate([15,-105,0])
enemy.translate([0.2,0.5,0])
enemy.scale([0.8,0.8,0.8])

player = Group([foot_left,foot_right,body])
player.scale([0.8,0.8,0.8],body.coord)
player.rotate([-15,15,-10],body.coord)
player.translate([0,0,-0.8])


while not glfw.window_should_close(window):
    
    #reset canvas
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)    
    glClearColor(1.0, 1.0, 1.0, 1.0)



    #representacao em malha
    if glfw.get_key(window, glfw.KEY_P) == glfw.PRESS and not p_pressed:
        is_Malha = not is_Malha
        p_pressed = True
    if glfw.get_key(window, glfw.KEY_P) == glfw.RELEASE:
        p_pressed = False
    if is_Malha:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    else: 
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)



    #chao
    chao.render()



    #enemy
    #muda de escala e cor com teclas x e z
    if glfw.get_key(window, glfw.KEY_Z) == glfw.PRESS and enemy_scale > 1: enemy_scale -= 0.01
    if glfw.get_key(window, glfw.KEY_X) == glfw.PRESS and enemy_scale < 1.5: enemy_scale += 0.01
    enemy.scale([enemy_scale,enemy_scale,enemy_scale])
    color = []
    for i in range(len(enemy_color)):
        color.append([])
        for j in range(4):
            color[i].append((enemy_color[i][j]*(-enemy_scale*2+3)+enemy_big_color[i][j]*(enemy_scale*2-2)))
    enemy.color = color
    enemy.render()
    enemy.scale([1/enemy_scale,1/enemy_scale,1/enemy_scale])
    
    enemy_frame+=1
    if enemy_frame == enemy_anim_frame:
        enemy_frame = 0
    enemy_traslation = math.cos(enemy_frame/enemy_anim_frame * 2*PI) * enemy_anim_speed
    enemy.translate([0,enemy_traslation,0])





    #player
    def cancel_squat():
        global state
        state = IDLE
        player.scale([1,1/0.5,1],sum_coord(body.coord,[0,-0.24,0]))

    def cancel_walk():
        global state, foot_right, foot_left, walk_rotate_sum,walk_frame
        state = IDLE
        if is_Right: 
            foot_right.rotate([0,0,-walk_init_rot - walk_rotate_sum])
            foot_left.rotate([0,0,+walk_init_rot + walk_rotate_sum])
        else: 
            foot_right.rotate([0,0,+walk_init_rot - walk_rotate_sum])
            foot_left.rotate([0,0,-walk_init_rot + walk_rotate_sum])
        walk_frame = 0
        walk_rotate_sum = 0

    #agachar
    if glfw.get_key(window, glfw.KEY_S) == glfw.PRESS and state != SQUAT and state != JUMP:
        if state == WALK:
            cancel_walk()
        state = SQUAT
        player.scale([1,0.5,1],sum_coord(body.coord,[0,-0.24,0]))
    if glfw.get_key(window, glfw.KEY_S) != glfw.PRESS and state == SQUAT:
        cancel_squat()

    #andar
    #velocidade
    player_velocity[0] = 0
    if glfw.get_key(window, glfw.KEY_A) == glfw.PRESS: player_velocity[0] -= player_x_velocity
    if glfw.get_key(window, glfw.KEY_D) == glfw.PRESS: player_velocity[0] += player_x_velocity
    if (body.coord[0] > 1 and player_velocity[0] > 0) or (body.coord[0] < -1 and player_velocity[0] < 0) or state == SQUAT:
        player_velocity[0] = 0
            
    #orientacao do player
    if is_Right and player_velocity[0] < 0 or not is_Right and player_velocity[0] > 0:
        is_Right = not is_Right
        player.scale([-1,1,1],body.coord)

    #animacao andar
    #start walk
    if player_velocity[0] != 0 and state != JUMP and state != WALK:
        state = WALK
        if is_Right: foot_right.rotate([0,0,walk_init_rot])
        else: foot_right.rotate([0,0,-walk_init_rot])
        if is_Right: foot_left.rotate([0,0,-walk_init_rot])
        else: foot_left.rotate([0,0,walk_init_rot])
    #walking
    if state == WALK:
        walk_frame += 1
        rotate_z = math.cos((walk_frame%walk_cicle_time * (1/walk_cicle_time)) * 2*PI) * walk_force
        if not is_Right: rotate_z = -rotate_z
        walk_rotate_sum += rotate_z
        foot_right.rotate([0,0,rotate_z])
        foot_left.rotate([0,0,-rotate_z])
        if player_velocity[0] == 0:
            cancel_walk()
    
        

    #pular
    #start jump
    if glfw.get_key(window, glfw.KEY_W) == glfw.PRESS and state != JUMP:
        if state == SQUAT:
            cancel_squat()
        if state == WALK:
            cancel_walk()
        state = JUMP
        jump_init_y = body.coord[1] 
        player_velocity[1] = jump_force
    #stop jump
    elif state == JUMP and body.coord[1] <= jump_init_y:
        state = IDLE
        body.translate([0,jump_init_y-body.coord[1],0])
        player_velocity[1] = 0
        jump_frame = 0
        if not is_Right:
            jump_rotate_sum_right = -jump_rotate_sum_right
            jump_rotate_sum_left = -jump_rotate_sum_left
        foot_right.rotate([0,0,-jump_rotate_sum_right])
        foot_left.rotate([0,0,-jump_rotate_sum_left])
        jump_rotate_sum_left = 0
        jump_rotate_sum_right = 0
    #jumping
    if state == JUMP:
        jump_frame+=1
        player_velocity[1] -= gravity
        if jump_frame <= jump_up_anim_frame:
            rotate_z_right = math.cos(jump_frame / jump_up_anim_frame * PI/2) * jump_right_up_rot
            rotate_z_left = math.cos(jump_frame / jump_up_anim_frame * PI/2) * jump_left_up_rot
            jump_rotate_sum_right += rotate_z_right
            jump_rotate_sum_left += rotate_z_left
            if not is_Right:
                rotate_z_right = -rotate_z_right
                rotate_z_left = -rotate_z_left
            foot_right.rotate([0,0,rotate_z_right])
            foot_left.rotate([0,0,rotate_z_left])



    #aplicar movimentacao
    player.translate(player_velocity)
    #render
    player.render()
    

    
    
    glfw.swap_buffers(window)
    glfw.poll_events()

glfw.terminate()